In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import psana
import h5py

import os
import sys

from center_finding import *

## Get x and y coordinates

In [ ]:
experiment='cxilw1118'
runNumber=12
# Set the installation and output path
os.environ['INSTALLPATH']= '/cds/home/i/igabalsk/TRXS-Run18'
os.environ['OUTPUTPATH']= '/cds/data/psdm/%s/%s/scratch' % (experiment[0:3],experiment)
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/LCLS')
from dataAnalysis import *
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/mattsLibraries')
from plotStyles import *
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/pythonBatchMagic')
from pythonBatchMagic import *

x,y = CSPADgeometry(detType='Jungfrau', run=runNumber, experiment=experiment)

# Load an image from PSANA

In [ ]:
# Create a data source to grab data from
ds = psana.DataSource('exp=%s:run=%d:smd' % (experiment , runNumber) )

data = None
nsum = 0
# Iterate through events
nevent = 0
evr = psana.Detector('evr1')
jf = psana.Detector('jungfrau4M')
for evt in ds.events():
    nevent+=1
    jfimg = jf.calib(evt,mbits=39)
    if jfimg is not None:
        jfimg[(jfimg<100.)|(jfimg>1000)] = 0.
        if data is None:
            data = jfimg
            nsum = 1
        else:
#             jfimg[jfimg<200]=0
#             jfimg[jfimg>200]=400
            data += jfimg
            nsum += 1
    if nevent%100==0:
        print nevent
    if nevent==1000: break
print 'Processed',nevent,'events.' 
# data = data/float(nsum)

## Plot raw image

In [ ]:
# %matplotlib notebook # Might need to be replaced by "%matplotlib inline" jupyterlab

plt.figure(figsize=(7,6))
for i in np.arange(8):
    plt.pcolormesh(x[i],y[i],data[i,:,:],vmin=0,vmax=1e3) # Color scale limits might need adjustment
plt.colorbar()
# plt.savefig('asymmetry-run%d.png' % runNumber)
plt.show()

## Mask off horizontal and vertical stripes

In [ ]:
mask = np.ones_like(data)
mask[:,255:257,:] = np.nan
mask[:,0:2,:] = np.nan
mask[:,511,:] = np.nan
mask[:,:,0] = np.nan
mask[:,:,255:257] = np.nan
mask[:,:,511:513] = np.nan
mask[:,:,767:769] = np.nan
mask[:,:,1023] = np.nan

# corrections_file = '/cds/home/i/igabalsk/TRXS-Run18/Libraries/Corrections.mat'

# corrections = {}
# with h5py.File(corrections_file, 'r') as f:
#     for key, val in f.items():
#         corrections[key] = np.array(val)
# print corrections.keys()
# for key in corrections.keys():
#     if corrections[key].shape==(1024,512,8):
#         corrections[key]=corrections[key].transpose(2,1,0)
#     elif corrections[key].shape==(1,8*512*1024):
#         corrections[key]=np.reshape(corrections[key], (1024,512,8)).transpose(2,1,0)
#     print key, corrections[key].shape

plt.figure(figsize=(7,6))
for i in np.arange(8):
    plt.pcolormesh(x[i],y[i],(data*mask)[i,:,:],vmin=0,vmax=1e4)
plt.colorbar()
plt.show()

masked_data = data*mask
masked_data[np.isnan(masked_data) | (masked_data<0) | (masked_data>1000)]=0
hists,bins = np.histogram(masked_data,bins=1000)
plt.figure()
plt.semilogy(bins[:-1],hists)
plt.show()

## Find center of diffraction pattern

In [ ]:
# The center is fitted here for a number of intensity ranges.

#Intensity ranges of the points for the center fitting, might need to be adjusted.
print np.nanmean(data), np.nanstd(data)

# CH2BrI
# lbounds = np.arange(6,12.5,0.5)
# ubounds = lbounds + 0.1
# Xe
# lbounds = np.arange(3,8.5,0.5)
# ubounds = lbounds + 0.1
# SF6
lbounds = np.arange(1,12,1)
ubounds = lbounds + 0.1
# CS2
# lbounds = np.arange(0.5,1.2,0.05)
# ubounds = lbounds + 0.005

xcenters = np.zeros_like(lbounds)
ycenters = np.zeros_like(lbounds)
for i in np.arange(len(lbounds)):
    xcenter,ycenter,inds = find_center(x,y,(data*mask/1e3),lbounds[i],ubounds[i])
    xcenters[i] = xcenter
    ycenters[i] = ycenter

In [ ]:
# Plot intensity range dependence of diffraction center:
plt.figure()
plt.plot(lbounds,xcenters,'.',label='Center x')
plt.plot(lbounds,ycenters,'.',label='Center y')
plt.xlabel('Intensity of pixels for ring fit')
plt.ylabel('Center Pixel')
plt.legend(loc='best')
plt.title('Dependence of fitted diffraction center on pixel intensity for ring-fitting')
plt.show()

In [ ]:
xcenter = np.mean(xcenters[1:-1])
ycenter = np.mean(ycenters[1:-1])
print xcenter,ycenter

## Load isotropic scattering from sample molecule (usually SF6)

In [ ]:
import h5py
from numpy.linalg import norm

molecule = 'SF6'
folder = '/cds/home/i/igabalsk/xray/diffraction_simulation/'
filepath = folder+'isotropic_scattering_%s.h5' % molecule

isotropic_scattering_dict = {}
with h5py.File(filepath,'r') as f:
    for name, _ in f.items():
        isotropic_scattering_dict[name]=f[name][()]

corrections_dict = {}
with h5py.File('/cds/home/i/igabalsk/TRXS-Run18/Libraries/Corrections_nightshift.mat','r') as f:
    for name, _ in f.items():
        corrections_dict[name]=f[name][()]
        print name, corrections_dict[name].shape
        
QQ_1d = isotropic_scattering_dict['QQ_1d']
isotropic_scattering_1d = isotropic_scattering_dict['isotropic_scattering_1d']
QQx, QQy, QQz = isotropic_scattering_dict['QQx'], isotropic_scattering_dict['QQy'], isotropic_scattering_dict['QQz']
QQ = isotropic_scattering_dict['QQ']

from scipy.interpolate import interp1d

ff = interp1d(QQ_1d,np.abs(isotropic_scattering_1d))
def form_factor(q):
    output = np.zeros_like(q)
    output[q>np.max(QQ_1d)] = ff(np.max(QQ_1d))
    output[q<np.min(QQ_1d)] = ff(np.min(QQ_1d))
    output[(q<=np.max(QQ_1d))&(q>=np.min(QQ_1d))] = ff(q[(q<=np.max(QQ_1d))&(q>=np.min(QQ_1d))])
    return output

qs = np.linspace(np.min(QQ_1d),np.max(QQ_1d),1000)
plt.figure()
plt.semilogy(qs, form_factor(qs))
plt.xlabel('Q',fontsize=15)
plt.ylabel('f(Q) isotropic',fontsize=15)
plt.title('Isotropic ensemble (%s)' % molecule, fontsize=20)
plt.show()

plt.figure(figsize=(7,6))
plt.pcolormesh(QQx,QQy,form_factor(norm(QQ,axis=-1)),
              norm=colors.LogNorm())
plt.colorbar()
plt.xlabel(r'$Q_x$',fontsize=15)
plt.ylabel(r'$Q_y$',fontsize=15)
plt.title('Isotropic ensemble (%s)' % molecule, fontsize=20)
plt.show()

## Define fitting function and helper functions

In [ ]:
# constants
joule_per_ev = 1.6e-19
photon_energy = 15.155e3
wavelength = 1240/photon_energy
alpha_theory = -np.log(0.94668)/500
print 'alpha =',alpha_theory

# helper functions
def theta_to_q(theta):
    return 4*np.pi*np.sin(theta/2.)/(wavelength*10)
def q_to_theta(q):
    return 2*np.arcsin((wavelength*10*q)/(4*np.pi))
def get_thomson_correction(x,y,z):
    r_xy = np.sqrt(x**2+y**2)
    theta = np.arctan(r_xy/z)
    phi = np.arctan2(y,x)
    correction = 1/(np.sin(phi)**2+np.cos(theta)**2*np.cos(phi)**2)
    return correction
def get_be_correction(x,y,z,alpha):
    r_xy = np.sqrt(x**2+y**2)
    theta = np.arctan(r_xy/z)
    be_correction = np.exp(alpha*theta_to_z(theta))
    return be_correction
def get_geometry_correction(x,y,z):
    # correction due to intensity falling off as 1/R**2
    R2 = (x**2+y**2+z**2)/z**2
    # correction due to flux through pixel area falling off with increased angle
    r_xy = np.sqrt(x**2+y**2)
    theta = np.arctan(r_xy/z)
    A= 1/np.cos(theta)
    return R2*A
def xyz_to_phi(x,y,z):
    return np.arctan2(y,x)

# function to use with curve_fit
def fitting_function(xy,xcenter,ycenter,z0,amplitude):
    x = np.ravel(xy[0])
    y = np.ravel(xy[1])
    xcent = x-xcenter
    ycent = y-ycenter
    r_xy = np.sqrt(xcent**2+ycent**2)
    theta = np.arctan(r_xy/z0)
    Q_abs = theta_to_q(theta)
    ff = amplitude*form_factor(Q_abs)
    thomson_correction = 1/get_thomson_correction(xcent,ycent,z0)
    geometry_correction = 1/get_geometry_correction(xcent,ycent,z0)
    ff = ff*thomson_correction*geometry_correction
    return ff

# converts angle to distance through beryllium
def theta_to_z(theta):
    return 500/np.cos(theta)
# function to use with curve_fit; alpha represents Be attenuation coefficient
def fitting_function_freealpha(xy,xcenter,ycenter,z0,amplitude,alpha):
    x = np.ravel(xy[0])
    y = np.ravel(xy[1])
    xcent = x-xcenter
    ycent = y-ycenter
    r_xy = np.sqrt(xcent**2+ycent**2)
    theta = np.arctan(r_xy/z0)
    Q_abs = theta_to_q(theta)
    ff = amplitude*form_factor(Q_abs)
    be_correction = np.exp(-alpha*theta_to_z(theta))
    be_correction = be_correction/np.max(be_correction)
    thomson_correction = 1/get_thomson_correction(xcent,ycent,z0)
    geometry_correction = 1/get_geometry_correction(xcent,ycent,z0)
    ff = ff*be_correction*thomson_correction*geometry_correction
    return ff

## Fit 2d pattern to functional form

In [ ]:
from scipy.optimize import curve_fit

x0, y0 = xcenter, ycenter
z0 = 93000.
alpha0 = 0.0006
amplitude = 0.1

# mask out area near optical axis
goodmask = np.ones_like(mask).astype(bool)
goodmask[np.isnan(mask)] = 0
goodmask[np.sqrt(x**2+y**2)<20000]=0

xgood = x[goodmask]
ygood = y[goodmask]
datagood = data[goodmask]
xy = [xgood,ygood]

params, covariances = curve_fit(fitting_function,xy,np.ravel(datagood),
                                p0=[0,0,z0,amplitude],
                                bounds=([-10000,-10000,30000,0],[10000,10000,100000,10000]))

In [ ]:
for item in params:
    print float(item)

In [ ]:
def xyz_to_q(x,y,z):
    r_xy = np.sqrt(x**2+y**2)
    theta = np.arctan(r_xy/z)
    Q_abs = theta_to_q(theta)
    return Q_abs

fit_pattern = np.reshape(fitting_function([x,y],*params),(8,512,1024))

QQ_data = xyz_to_q(x-params[0],y-params[1],params[2])
isotropic_data_1d = []
isotropic_fit_1d = []
QQ_1d_data = np.linspace(np.min(QQ_data),np.max(QQ_data),51)
dQQ_1d_data = np.mean(np.diff(QQ_1d_data))
QQ_1d_data = QQ_1d_data[:-1]
for qval in QQ_1d_data:
    isotropic_data_1d.append(np.mean(data[(QQ_data>qval)&(QQ_data<qval+dQQ_1d_data)]))
    isotropic_fit_1d.append(np.mean(fit_pattern[(QQ_data>qval)&(QQ_data<qval+dQQ_1d_data)]))

# with h5py.File('SF6data_for_Matt.h5','w') as f:
#     f.create_dataset('QQ_1d',data=QQ_1d_data)
#     f.create_dataset('isotropic_data_1d',data=isotropic_data_1d)
#     f.create_dataset('isotropic_fit_1d',data=isotropic_fit_1d)
#     f.create_dataset('params',data=params)

plt.figure(figsize=(9,6))
plt.semilogy(QQ_1d_data,isotropic_data_1d,'.',label='Data',markersize=12)
plt.semilogy(QQ_1d_data,isotropic_fit_1d,label='Fit',linewidth=3)
# plt.plot(QQ_1d_data, form_factor(QQ_1d_data))
plt.legend(fontsize=15)
plt.rc('font',size=20)
ax = plt.gca()
ax.tick_params(which='major',width=2, length=8)
ax.tick_params(which='minor',width=1, length=5)
plt.xlabel('Q (inv. Ang)')
plt.ylabel('Intensity (arb)')
plt.title('%s Calibration Run %d' % (molecule, runNumber))
# plt.savefig('SF6_calibration_%.0f_%.3f_%.3f.png' % (params[0],params[1],params[2]), bbox_inches='tight')
plt.show()

In [ ]:
dummydata = np.copy(data)
dummydata[~goodmask]=0
plt.figure(figsize=(7,6))
for i in np.arange(8):
    plt.pcolormesh(x[i],y[i],(dummydata)[i,:,:],vmin=0,vmax=np.max(fit_pattern)/10)
plt.colorbar()
plt.show()

plt.figure(figsize=(7,6))
for i in np.arange(8):
    plt.pcolormesh(x[i],y[i],(fit_pattern)[i,:,:],vmin=0,vmax=np.max(fit_pattern)/10)
plt.colorbar()
plt.show()

## Generate Q map and Thomson correction file

In [ ]:
thomson_correction = get_thomson_correction(x-params[0],y-params[1],params[2])
PHI = xyz_to_phi(x-params[0],y-params[1],params[2])
geometry_correction = get_geometry_correction(x-params[0],y-params[1],params[2])

with h5py.File('/cds/home/i/igabalsk/TRXS-Run18/Libraries/lu92_calibrations.h5','w') as f:
    f.create_dataset('QQ',data=QQ_data)
    f.create_dataset('PHI',data=PHI)
    f.create_dataset('thomson_correction',data=thomson_correction)
    f.create_dataset('geometry_correction',data=geometry_correction)

plt.figure(figsize=(7,6))
for i in np.arange(8):
    plt.pcolormesh(x[i],y[i],(thomson_correction)[i,:,:],vmin=1,vmax=1.8)
plt.colorbar()
plt.title('Thomson correction')
plt.show()

plt.figure(figsize=(7,6))
for i in np.arange(8):
    plt.pcolormesh(x[i],y[i],(geometry_correction)[i,:,:],vmin=1,vmax=np.max(geometry_correction))
plt.colorbar()
plt.title('Geometry correction')
plt.show()
print 'Qmax=',np.max(QQ_data)

In [ ]:
plt.figure(figsize=(7,6))
for i in np.arange(8):
    plt.pcolormesh(x[i],y[i],(QQ_data)[i,:,:],vmin=0,vmax=np.max(QQ_data))
plt.colorbar()
plt.title('Q map')
plt.show()
plt.figure(figsize=(7,6))
for i in np.arange(8):
    plt.pcolormesh(x[i],y[i],(PHI)[i,:,:],vmin=-np.pi,vmax=np.pi)
plt.colorbar()
plt.title('Phi map')
plt.show()

### Generate binning indices using Q, phi maps

In [ ]:
print np.min(QQ), np.max(QQ)

In [ ]:
QQ = QQ_data

Qmin, Qmax = 0.5, 4.5
NQ = 100
Qs = np.linspace(Qmin,Qmax,NQ)
dQ = np.mean(np.diff(Qs))

Nphi = 16
phis = np.linspace(0,np.pi/2-(np.pi/2/Nphi),Nphi)
dphi = np.mean(np.diff(phis))

roi_indices = []

for qidx, q in enumerate(Qs):
    print 'Qidx: ',qidx
    qrow = []
    for phiidx, phi in enumerate(phis):
        qphibin_mask = np.zeros((8,512,1024)).astype(bool)
        qphibin_mask = qphibin_mask | ((QQ>=q)&(QQ<q+dQ)&(PHI>=phi)&(PHI<phi+dphi)&(~np.isnan(mask)))
        qphibin_mask = qphibin_mask | ((QQ>=q)&(QQ<q+dQ)&(PHI<=-phi)&(PHI>-phi-dphi)&(~np.isnan(mask)))
        qphibin_mask = qphibin_mask | ((QQ>=q)&(QQ<q+dQ)&(PHI>=np.pi-phi-dphi)&(PHI<np.pi-phi)&(~np.isnan(mask)))
        qphibin_mask = qphibin_mask | ((QQ>=q)&(QQ<q+dQ)&(PHI>=phi-np.pi)&(PHI<phi-np.pi+dphi)&(~np.isnan(mask)))
        qrow.append(np.nonzero(qphibin_mask))
    roi_indices.append(qrow)
binning_indices = BinningIndices(Qs,dQ,phis,dphi,roi_indices)
save_obj(binning_indices,'/cds/home/i/igabalsk/TRXS-Run18/Libraries/binning_indices')

In [ ]:
dummy = np.zeros((8,512,1024))
n = 1
for phiIdx in range(Nphi):
    for qIdx in range(NQ/(2*n)):
        if qIdx%2==0:
            for j in range(n):
                dummy[roi_indices[2*n*qIdx+j][phiIdx]]=1
                dummy[roi_indices[2*n*qIdx+j+n][phiIdx]]=-1
        else:
            for j in range(n):
                dummy[roi_indices[2*n*qIdx+j][phiIdx]]=-1
                dummy[roi_indices[2*n*qIdx+j+n][phiIdx]]=1
# phiIdx = 6
# for qIdx in range(0,25):
#     dummy[roi_indices[qIdx][phiIdx]]=1
# for qIdx in range(25,50):
#     dummy[roi_indices[qIdx][phiIdx]]=-1
# for qIdx in range(50,75):
#     dummy[roi_indices[qIdx][phiIdx]]=1
# for qIdx in range(75,100):
#     dummy[roi_indices[qIdx][phiIdx]]=-1

In [ ]:
plt.figure(figsize=(10,8))
for i in np.arange(8):
    plt.pcolormesh(x[i],y[i],(dummy)[i,:,:],vmin=-1,vmax=1,cmap=plt.get_cmap('viridis'))
plt.colorbar()
plt.title(r'$(Q,\phi)$ bins',fontsize=30)
plt.show()
print np.max(Qs)+np.mean(np.diff(Qs)), np.max(QQ_data)

### Generate radial ROI binning indices

In [ ]:
RR = np.sqrt((x+1206)**2+(y+461)**2)

Rmin, Rmax = 0, 1e5
NR = 200
Rs = np.linspace(Rmin,Rmax*(1-1./200),NR)
dR = np.mean(np.diff(Rs))

roi_indices = []

for ridx, r in enumerate(Rs):
    rrow = np.nonzero((RR>=r)&(RR<r+dR)&(~np.isnan(mask)))
    print 'Ridx: ',ridx, np.array(rrow).shape
    roi_indices.append(rrow)
radial_binning_indices = BinningIndices(Rs,dR,[0],0,roi_indices)
save_obj(radial_binning_indices,'/cds/home/i/igabalsk/TRXS-Run18/Libraries/radial_binning_indices')

In [ ]:
dummy = np.zeros((8,512,1024))
n = 1
val = -1
for ri in range(NR):
    if ri%n==0:
        val *= -1
    dummy[roi_indices[ri]]=val

In [ ]:
plt.figure(figsize=(10,8))
for i in np.arange(8):
    plt.pcolormesh(x[i],y[i],(dummy)[i,:,:],vmin=-1,vmax=1,cmap=plt.get_cmap('viridis'))
plt.colorbar()
plt.title(r'$(Q,\phi)$ bins',fontsize=30)
plt.show()

In [ ]:
nate_mask_old = np.load('/cds/data/psdm/cxi/cxilv1118/results/nate/Mask_Jungfrau_57_58.npy')
nate_mask_new = np.load('/cds/data/psdm/cxi/cxilv1118/results/nate/Mask_Jungfrau_107_109.npy')
print float(len(nate_mask_old[nate_mask_old==1]))/len(nate_mask_old.flatten())
print float(len(nate_mask_new[nate_mask_new==1]))/len(nate_mask_new.flatten())

In [ ]:
print nate_mask_new.shape
from scipy.ndimage import gaussian_filter
gf = lambda x: gaussian_filter(x,1)
plt.figure(figsize=(10,8))
for i in np.arange(8):
    plt.pcolormesh(x[i],y[i],(1-nate_mask_old)[i,:,:],vmin=0,vmax=1,cmap=plt.get_cmap('viridis'))
plt.colorbar()
plt.title(r'$(Q,\phi)$ bins',fontsize=30)
plt.show()

In [ ]:
plt.figure()
plt.hist(nate_mask_new.flatten(),bins=100)
plt.show()
print np.sum(nate_mask_new)/float(nate_mask_new.size)